In [ ]:
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

In [ ]:
!pip install groq

In [ ]:
import os
from groq import Groq
import json

# Initialize Groq Client
client = Groq(api_key="GROQ API KEY HERE")  # Replace with your actual API key

scenarios = [line.strip() for line in open("data/scenarios.txt", "r") if line.strip()]

def generate_groq_trace(scenario):
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are a senior SRE expert. Generate a technical action-trace. "
                               "The 'action' field must be a string containing the exact CLI commands. "
                               "The 'correction' field must be a string with a safety tip."
                },
                {
                    "role": "user",
                    "content": f"Generate a JSON object for: {scenario}. Keys: instruction, thought, action, observation, correction."
                }
            ],
            model="llama-3.3-70b-versatile",
            response_format={"type": "json_object"}
        )

        # Parse the JSON to ensure it's valid, then dump it as a single line
        raw_content = chat_completion.choices[0].message.content
        data = json.loads(raw_content)
        return json.dumps(data) # This removes all newlines and pretty-printing
    except Exception as e:
        print(f"Error generating for {scenario}: {e}")
        return None

# Save as a proper JSONL file
with open("git_traces_fixed.jsonl", "w") as f:
    for i, s in enumerate(scenarios):
        print(f"[{i+1}/20] Generating: {s[:40]}...")
        trace = generate_groq_trace(s)
        if trace:
            f.write(trace + "\n")

print("\n✅ Dataset generated successfully: git_traces_fixed.jsonl")

In [ ]:
with open("git_traces_fixed.jsonl", "r") as f:
    lines = f.readlines()
    print(f"Total rows: {len(lines)}")
    print(f"First row check: {lines[0][:100]}...")

In [ ]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit", # Our base model
    max_seq_length = 2048,
    load_in_4bit = True,
)

print("Model and Tokenizer loaded successfully! You are ready to fine-tune.")

In [ ]:
from datasets import load_dataset
from unsloth import FastLanguageModel
import torch

# 1. Load the fixed dataset
dataset = load_dataset("json", data_files="git_traces_fixed.jsonl", split="train")

# 2. Define the Prompt Template
git_prompt_style = """### Instruction:
{}

### Thought (SRE Reasoning):
{}

### Action (CLI Command):
{}

### Observation (Terminal Output):
{}

### Correction/Safety Tip:
{}"""

EOS_TOKEN = tokenizer.eos_token 

def format_prompts(examples):
    instructions = examples["instruction"]
    thoughts     = examples["thought"]
    actions      = examples["action"]
    observations = examples["observation"]
    corrections  = examples["correction"]
    texts = []
    for inst, th, act, obs, corr in zip(instructions, thoughts, actions, observations, corrections):
        # Format the fields into the template
        text = git_prompt_style.format(inst, th, act, obs, corr) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

dataset = dataset.map(format_prompts, batched = True)
print(f"Dataset prepared! Total rows: {len(dataset)}")

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Rank: higher = more parameters, lower = faster
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Optimized to 0 for Unsloth
    bias = "none",    # Optimized to "none" for Unsloth
    use_gradient_checkpointing = "unsloth", # Saves memory
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

print("LoRA Adapters added. Model is now trainable!")

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 1e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        output_dir = "outputs",
    ),
)

trainer.train()

In [ ]:
model.save_pretrained("git_sre_lora_model")
tokenizer.save_pretrained("git_sre_lora_model")

In [ ]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "git_sre_lora_model", # Path to your saved folder
    max_seq_length = 2048,
    load_in_4bit = True,
)


In [ ]:
# Switch to inference mode
FastLanguageModel.for_inference(model)

test_prompt = """### Instruction:
I pushed my .env file with all api keys. How can I remove it from history

### Thought (SRE Reasoning):"""

inputs = tokenizer([test_prompt], return_tensors = "pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens = 150, repetition_penalty = 1.2)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
def compare_models(instruction):
    prompt = f"### Instruction:\n{instruction}\n\n### Thought (SRE Reasoning):"
    inputs = tokenizer([prompt], return_tensors = "pt").to("cuda")

    # 1. Test your Fine-Tuned Model
    FastLanguageModel.for_inference(model)
    ft_outputs = model.generate(**inputs, max_new_tokens = 150)
    ft_result = tokenizer.decode(ft_outputs[0], skip_special_tokens=True)

    # 2. Test the Base Model (Disable adapters temporarily)
    with model.disable_adapter():
        base_outputs = model.generate(**inputs, max_new_tokens = 150)
        base_result = tokenizer.decode(base_outputs[0], skip_special_tokens=True)

    print(f"--- BASE MODEL ---\n{base_result}\n")
    print(f"--- YOUR MODEL ---\n{ft_result}\n")

compare_models("I force-pushed to main and lost my coworker's work.")